# Model Evaluation

This notebook is to evaluate a selected checkpoint (from a list of save checkpoints at different number of training iterations during RL model training) on the training dataset.

This notebook is used to evaluate trained PPO-MI models.

In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
from datetime import datetime
import sys

from utils import print_log

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# load the created dataset
dataset_folder_path = Path("dataset", "20250707_downsampled_1min", "split")

In [4]:
from utils_data_split import load_split_data_from_folder, convert_to_naive_datetimes, convert_to_naive_datetimes_df

In [5]:
aggregate_load_segments_train, aggregate_load_df_train = load_split_data_from_folder(dataset_folder_path, 'train') 

In [6]:
aggregate_load_segments_train, aggregate_load_df_train = convert_to_naive_datetimes(aggregate_load_segments_train), convert_to_naive_datetimes_df(aggregate_load_df_train )

---

In [ ]:
# initialize experiment folder to store related information/data for future analysis

# Change the datetime to point to the correct folder
# rl_datetime = datetime(2025, 8, 11, 2, 46, 31)
rl_datetime = datetime(2025, 8, 11, 4, 45, 55)
# rl_datetime = datetime(2025, 8, 11, 6, 42, 39)

# Keep the action type un-changed
ACTION_TYPE = "continuous"

# Change the lambda according to your experiment
REWARD_LAMBDA = "0.5"  # between 0 and 1

experiment_folder = Path(
    "experiments", rl_datetime.strftime('%Y%m%d_%H%M%S') + f"_action_{ACTION_TYPE}_reward_lambda_{REWARD_LAMBDA}", 
)

if not experiment_folder.exists():
    raise FileNotFoundError(f"Experiment folder {experiment_folder} does not exist. Please check the path.")
else:
    print_log(f"Experiment folder: {experiment_folder}")

[2025-09-23 16:10:26:493] Experiment folder: experiments/20250811_044555_action_continuous_reward_lambda_0.5


In [8]:
# find the selected checkpoint

selected_ckpt_folder = experiment_folder / "selected_ckpt"

# search for any sub-folders under selected_ckpt_folder
sub_folders = [f for f in selected_ckpt_folder.iterdir() if f.is_dir()]
# assert len(sub_folders) == 1, "There should be exactly one sub-folder in selected_ckpt_folder."

selected_ckpt_folder = sub_folders[0]
selected_ckpt_episode_number = int(selected_ckpt_folder.name.split('_')[-1])

print_log(f"Selected checkpoint folder: {selected_ckpt_folder}")

[2025-09-23 16:10:26:508] Selected checkpoint folder: experiments/20250811_044555_action_continuous_reward_lambda_0.5/selected_ckpt/episode_0602


We create a new folder under the experiment_folder specifically for storing the results of executing a subset of training episodes with the selected ckpt (for IEEE journal)

In [9]:
log_folder = experiment_folder / "logs_train_selected-ckpt"

if not log_folder.exists():
    log_folder.mkdir()

Create data loader

In [10]:
sys.path.append(str(Path('rl_env')))
sys.path.append(str(Path('model', 'H_network')))

from rl_env.data_loader import SimpleSmartMeterDataLoader

sm_dl_train = SimpleSmartMeterDataLoader(
    aggregate_load_segments=aggregate_load_segments_train,
    aggregate_load_df=aggregate_load_df_train,
    registry_path=log_folder / "simple_episode_registry_train_subset.json"
)

sm_dl_train.get_divided_segments_length()

[SimpleDataLoader] Generated 162 episodes (1 day each)


162

Create environment

In [11]:
import torch
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
from model.H_network.common.factories import create_h_network_module_with_defaults
from model.H_network.h_network_arch import HNetworkType

# the optimizer class and optimizer kwargs does not matter for the testing loop, but they are required to create the module

h_network_kwargs = {
    "h_network_type": HNetworkType.H_NETWORK,
    "optimizer_class": torch.optim.Adam,
    "optimizer_kwargs": {"lr": 0.001},
}

h_network_rl_module = create_h_network_module_with_defaults(
    action_type=ACTION_TYPE,
    **h_network_kwargs,
    device=DEVICE
)

In [13]:
from rl_env.base.env_module import SmartMeterEnvFactory
from rl_env.training_mode import TrainingMode
from decimal import Decimal

env_train = SmartMeterEnvFactory.create(
    action_type=ACTION_TYPE,
    smart_meter_data_loader=sm_dl_train,
    h_network_rl_module=h_network_rl_module,
    mode=TrainingMode.TEST,
    reward_lambda=float(REWARD_LAMBDA),
    render_mode=None,
)

[2025-09-23 16:10:27:682] [SmartMeterContinuousEnv Test] Using data loader: SimpleSmartMeterDataLoader
[2025-09-23 16:10:27:682] [SmartMeterContinuousEnv Test] Curriculum enabled: False
[2025-09-23 16:10:27:688] [SmartMeterContinuousEnv Test]] Render mode set to 'None'. Render server at 127.0.0.1:50007. render_connected: False. render_client_socket: None


In [14]:
h_network_rl_module.set_h_network(
    h_network_rl_module.initialize_h_network()
)
h_network_rl_module.initialize_h_network_training()

In [15]:
# load the h_network from the selected checkpoint

h_network_rl_module.load_h_network(
    selected_ckpt_folder / ("h_network" + f"_{selected_ckpt_episode_number:04d}" + ".pth")
)

In [16]:
h_network_rl_module.h_network

HNetwork(
  (LSTM_1): LSTM(2, 44, batch_first=True, bidirectional=True)
  (ac1): Tanh()
  (LSTM_2): LSTM(88, 44, batch_first=True, bidirectional=True)
  (ac2): Tanh()
  (fc): Linear(in_features=88, out_features=1, bias=True)
)

In [17]:
import torchinfo

torchinfo.summary(h_network_rl_module.h_network, input_size=(1, 1, 2))

Layer (type:depth-idx)                   Output Shape              Param #
HNetwork                                 [1, 1]                    --
├─LSTM: 1-1                              [1, 1, 88]                16,896
├─Tanh: 1-2                              [1, 1, 88]                --
├─LSTM: 1-3                              [1, 1, 88]                47,168
├─Tanh: 1-4                              [1, 1, 88]                --
├─Linear: 1-5                            [1, 1, 1]                 89
Total params: 64,153
Trainable params: 64,153
Non-trainable params: 0
Total mult-adds (M): 0.06
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.26
Estimated Total Size (MB): 0.26

In [18]:
from gymnasium.utils.env_checker import check_env

# This will catch many common issues
try:
    check_env(env_train)
    print("Environment passes all checks!")
except Exception as e:
    print(f"Environment has issues: {e}")

[2025-09-23 16:10:28:487] [SmartMeterContinuousEnv Test] Resetting environment with episode 82. Episode info: {'length': 1440, 'datetime_range': (Timestamp('2013-08-31 00:00:01'), Timestamp('2013-08-31 23:59:01'))}
[2025-09-23 16:10:28:491] [SmartMeterContinuousEnv Test] Resetting environment with episode 2. Episode info: {'length': 693, 'datetime_range': (Timestamp('2013-01-10 00:00:02'), Timestamp('2013-01-10 11:32:02'))}
[2025-09-23 16:10:28:494] [SmartMeterContinuousEnv Test] Resetting environment with episode 110. Episode info: {'length': 1440, 'datetime_range': (Timestamp('2013-09-26 00:00:00'), Timestamp('2013-09-26 23:59:00'))}
[2025-09-23 16:10:28:497] [SmartMeterContinuousEnv Test] Resetting environment with episode 2. Episode info: {'length': 693, 'datetime_range': (Timestamp('2013-01-10 00:00:02'), Timestamp('2013-01-10 11:32:02'))}
[2025-09-23 16:10:28:500] [SmartMeterContinuousEnv Test] Resetting environment with episode 110. Episode info: {'length': 1440, 'datetime_range

/root/miniforge3/envs/smartmeter/lib/python3.9/site-packages/gymnasium/utils/env_checker.py:434: UserWarning: WARN: Not able to test alternative render modes due to the environment not having a spec. Try instantiating the environment through `gymnasium.make`
  logger.warn(


In [19]:
env_train.reset_render_window()

In [ ]:
# initialize a PPO agent
from stable_baselines3 import PPO

rl_model = PPO.load(
    selected_ckpt_folder / ("rl_model" + f"_{selected_ckpt_episode_number:04d}"),  # path to the saved RL model
    env=env_train
)

2025-09-23 16:10:28.914016: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-23 16:10:28.922782: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758615028.933502  119929 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758615028.936462  119929 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758615028.943230  119929 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [21]:
rl_model.policy

MultiInputActorCriticPolicy(
  (features_extractor): CombinedExtractor(
    (extractors): ModuleDict(
      (aggregate_load): Flatten(start_dim=1, end_dim=-1)
      (battery_soc): Flatten(start_dim=1, end_dim=-1)
      (timestamp_features): Flatten(start_dim=1, end_dim=-1)
    )
  )
  (pi_features_extractor): CombinedExtractor(
    (extractors): ModuleDict(
      (aggregate_load): Flatten(start_dim=1, end_dim=-1)
      (battery_soc): Flatten(start_dim=1, end_dim=-1)
      (timestamp_features): Flatten(start_dim=1, end_dim=-1)
    )
  )
  (vf_features_extractor): CombinedExtractor(
    (extractors): ModuleDict(
      (aggregate_load): Flatten(start_dim=1, end_dim=-1)
      (battery_soc): Flatten(start_dim=1, end_dim=-1)
      (timestamp_features): Flatten(start_dim=1, end_dim=-1)
    )
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=5, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=Tr

In [22]:
rl_model.n_steps

704

We execute on all training episodes

In [23]:
total_episodes = env_train.smart_meter_data_loader.get_divided_segments_length()
total_episodes

162

In [24]:
episode_indices = list(range(total_episodes))

In [25]:
# we test the rl model and H-network

import json
from utils import print_log


all_episode_rewards = []
all_episode_lengths = []
all_episode_infos = []
per_episode_reward_stats = []

for i, episode_idx in enumerate(episode_indices):
    obs, info = env_train.reset(episode_idx=episode_idx)

    # Initialize episode tracking variables
    episode_reward = 0.0
    episode_length = 0
    done = False
    state = None  # For recurrent policies

    while not done:
        # Get action from the RL model
        # no deterministic, as it will break the learned stochastic policy
        # think of dropout-layers in text-to-voice models
        action, state = rl_model.predict(obs, state=state)

        # Step the environment
        obs, reward, done, truncated, info = env_train.step(action)

        # Update episode tracking variables
        episode_reward += reward
        episode_length += 1

    # Store results for this episode
    all_episode_rewards.append(episode_reward)
    all_episode_lengths.append(episode_length)
    all_episode_infos.append({
        "episode_idx": episode_idx,
        "episode_info": env_train.episode.get_episode_info() if hasattr(env_train, 'episode') else {}
    })

    per_episode_reward_stats.append(env_train.prev_episode_reward_stats)

    env_train.save_episode_info(
        log_folder=log_folder,
        episode_training_idx=episode_idx
    )

    if (i + 1) % 10 == 0:
        print_log(f"Episode {i + 1}/{total_episodes} completed.")


# Calculate comprehensive statistics
mean_reward = np.mean(all_episode_rewards)
std_reward = np.std(all_episode_rewards)
min_reward = np.min(all_episode_rewards)
max_reward = np.max(all_episode_rewards)
median_reward = np.median(all_episode_rewards)

# we need to recover the true sum f_signal and g_signal from the per_episode_reward_stats, with length of each episode
f_signal_sums = np.array([item['f_signal_sum'] for item in per_episode_reward_stats])
g_signal_sums = np.array([item['g_signal_sum'] for item in per_episode_reward_stats])

# get the length of each episode
_episode_lengths = np.array(all_episode_lengths)
sum_episode_lengths = np.sum(_episode_lengths)

# calculate the (true) mean of rewards, f_signal and g_signal

f_signal_mean = f_signal_sums @ _episode_lengths / sum_episode_lengths if sum_episode_lengths > 0 else 0.0
g_signal_mean = g_signal_sums @ _episode_lengths / sum_episode_lengths if sum_episode_lengths > 0 else 0.0

mean_length = np.mean(all_episode_lengths)
std_length = np.std(all_episode_lengths)

# Prepare validation data
test_data = {
    'evaluated_episodes': len(episode_indices),
    'total_available_episodes': total_episodes,
    'episode_indices': episode_indices,
    'episode_rewards': all_episode_rewards,
    'episode_lengths': all_episode_lengths,
    'episode_infos': all_episode_infos,
    'summary_stats': {
        'mean_reward': mean_reward,
        'std_reward': std_reward,
        'min_reward': min_reward,
        'max_reward': max_reward,
        'median_reward': median_reward,
        'mean_f_signal': f_signal_mean,
        'mean_g_signal': g_signal_mean,
        'mean_length': mean_length,
        'std_length': std_length
    }
}

test_stats_file_path = log_folder / "train-selected-ckpt_stats_file.json"
with open(test_stats_file_path, 'w') as f:
    # Convert numpy types to Python native types for JSON serialization
    def convert_numpy(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, list):
            return [convert_numpy(item) for item in obj]
        elif isinstance(obj, dict):
            return {key: convert_numpy(value) for key, value in obj.items()}
        return obj

    json.dump(convert_numpy(test_data), f, indent=2, default=str)

sm_dl_train._save_registry()
print_log(f"Validation results saved to {test_stats_file_path}")

[2025-09-23 16:10:30:122] [SmartMeterContinuousEnv Test] Resetting environment with episode 0. Episode info: {'length': 1440, 'datetime_range': (Timestamp('2013-01-08 00:00:05'), Timestamp('2013-01-08 23:59:05'))}


[2025-09-23 16:10:35:709] [SmartMeterContinuousEnv Test] Episode finished. Sum of rewards: 138.79212788707366. Mean of rewards: 0.09645040158934931. Std of rewards: 0.06748258138540024
[2025-09-23 16:10:35:709] [SmartMeterContinuousEnv Test] Episode f_signal sum: -282.01390248151483. Mean: -0.19597908442078862. Std: 0.13584807016978712
[2025-09-23 16:10:35:709] [SmartMeterContinuousEnv Test] Episode g_signal sum: 4.429646707367466. Mean: 0.003078281242089969. Std: 0.0025107003280679737
[2025-09-23 16:10:35:726] [SmartMeterContinuousEnv Test] Episode 0000 info saved to experiments/20250811_044555_action_continuous_reward_lambda_0.5/logs_train_selected-ckpt/episode_info/episode_0000_info.json
[2025-09-23 16:10:35:730] [SmartMeterContinuousEnv Test] Resetting environment with episode 1. Episode info: {'length': 1440, 'datetime_range': (Timestamp('2013-01-09 00:00:04'), Timestamp('2013-01-09 23:59:04'))}
[2025-09-23 16:10:41:464] [SmartMeterContinuousEnv Test] Episode finished. Sum of rewa